In [1]:
import torch
import os
import sys
sys.path.append(os.path.abspath(".."))

from ase import Atoms
from ase.visualize import view
from ase.build import bulk
from ase.units import GPa
from ase.spacegroup import crystal

from mattersim.applications.batch_relax import BatchRelaxer
from mattersim.forcefield.potential import Potential
from mattersim.datasets.utils.build import build_dataloader
from mattersim.forcefield.potential import MatterSimCalculator
from mattersim.applications.relax import Relaxer

from utils.visualisation import plot_potential, plot_relaxation, visualise_structure

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"MatterSim running on {device}")

MatterSim running on cuda


In [3]:
model = 5

In [4]:
# Define the L1₀ unit cell parameters
a = 3.85  # Lattice constant in x and y
c = 3.72  # Lattice constant in z (slightly compressed)
alpha, beta, gamma = 90, 90, 90

# Define the L1₀ structure using ASE
structure = crystal(
    symbols=['Fe', 'Pt'],
    basis=[[0, 0, 0], [0.5, 0.5, 0.5]],  # Atomic positions as fractional coordinates
    spacegroup=123,  # P4/mmm
    cellpar=[a, a, c, alpha, beta, gamma]  # a, b, c, alpha, beta, gamma
)

In [5]:
_ = visualise_structure(structure)

Structure positions: [[0.    0.    0.   ]
 [1.925 1.925 1.86 ]]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [6]:
structure.calc = MatterSimCalculator(load_path=f"MatterSim-v1.0.0-{model}M.pth", device=device)

2025-02-05 06:46:58.996 | INFO     | mattersim.forcefield.potential:from_checkpoint:891 - Loading the pre-trained mattersim-v1.0.0-5M.pth model
2025-02-05 06:46:59.971 | INFO     | mattersim.forcefield.potential:from_checkpoint:891 - Loading the pre-trained mattersim-v1.0.0-5M.pth model


/usr/local/lib/python3.12/dist-packages/mattersim/forcefield/potential.py:896: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_path, map_location=

In [7]:
print(f"Energy (eV)                 = {structure.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {structure.get_potential_energy()/len(structure)}")
print(f"Forces of first atom (eV/A) = {structure.get_forces()[0]}")
print(f"Stress[0][0] (x-x) (eV/A^3) = {structure.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (x-x) (GPa)    = {structure.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -10.912311553955078
Energy per atom (eV/atom)   = -5.456155776977539
Forces of first atom (eV/A) = [ 9.7602606e-07  1.3038516e-06 -1.3113022e-06]
Stress[0][0] (x-x) (eV/A^3) = 0.1328575922039818
Stress[0][0] (x-x) (GPa)    = 21.2861328125


In [8]:
# initialize the relaxation object
relaxer = Relaxer(
    optimizer="BFGS", # the optimization method
    filter = "FrechetCellFilter",
    # filter="ExpCellFilter"
    # filter = None, # filter to apply to the cell
    constrain_symmetry=True, # whether to constrain the symmetry
)
pressure_in_GPa=0
relaxed_structure = relaxer.relax(structure, params_filter={"scalar_pressure": pressure_in_GPa}, steps=500)

      Step     Time          Energy          fmax
BFGS:    0 06:46:59      -10.912312        3.662864
BFGS:    1 06:46:59      -11.506659        4.003235
BFGS:    2 06:46:59      -13.788523        3.172477
BFGS:    3 06:46:59      -14.626232        0.188771
BFGS:    4 06:46:59      -14.628473        0.195118
BFGS:    5 06:46:59      -14.189611        2.630440
BFGS:    6 06:46:59      -14.635093        0.220789
BFGS:    7 06:46:59      -14.641842        0.307623
BFGS:    8 06:46:59      -14.381277        2.273961
BFGS:    9 06:46:59      -14.655121        0.487818
BFGS:   10 06:46:59      -14.663462        0.613781
BFGS:   11 06:46:59      -14.667635        0.853699
BFGS:   12 06:46:59      -14.680408        0.721863
BFGS:   13 06:46:59      -14.690158        0.622398
BFGS:   14 06:46:59      -14.708532        0.356019
BFGS:   15 06:46:59      -14.714393        0.097587
BFGS:   16 06:46:59      -14.714853        0.013357
BFGS:   17 06:46:59      -14.714863        0.001912


In [9]:
structure.get_positions()

array([[0.        , 0.        , 0.        ],
       [1.62015503, 1.62015503, 1.37549668]])

In [10]:
_ = visualise_structure(structure)

Structure positions: [[0.         0.         0.        ]
 [1.62015503 1.62015503 1.37549668]]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [11]:
# Define the L1₀ unit cell parameters
a = 3.85  # Lattice constant in x and y
c = 3.72  # Lattice constant in z (slightly compressed)

# Define the atomic positions and symbols
positions = [
    (0, 0, 0),               # Fe atom
    (0.5 * a, 0.5 * a, 0.5 * c),  # Pt atom
]

symbols = ['Fe', 'Pt']

# Create the unit cell for L1₀ FePt
structure = Atoms(symbols="FePt",
                 positions=positions,
                 cell=[(a, 0, 0), (0, a, 0), (0, 0, c)],
                 pbc=True)  # Periodic boundary conditions

structure.rattle(stdev=0.01)

In [12]:
# # Define the L1₀ unit cell parameters
# a = 3.85  # Lattice constant in x and y
# c = 3.72  # Lattice constant in z (slightly compressed)
# alpha, beta, gamma = 90, 90, 90


# # Define the L1₀ structure using spacegroup
# structure = crystal(
#     symbols=['Fe', 'Pt'],
#     basis=[[0, 0, 0], [0.5, 0.5, 0.5]],  # Atomic positions as fractional coordinates
#     spacegroup=123,  # P4/mmm
#     cellpar=[a, a, c, alpha, beta, gamma]  # a, b, c, alpha, beta, gamma
# )

# structure.rattle(stdev=0.01)

In [13]:
structure.get_scaled_positions()

array([[0.00129017, 0.99964087, 0.0017411 ],
       [0.50395592, 0.49939181, 0.4993706 ]])

In [14]:
_ = visualise_structure(structure)

Structure positions: [[ 4.96714153e-03 -1.38264301e-03  6.47688538e-03]
 [ 1.94023030e+00  1.92265847e+00  1.85765863e+00]]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [15]:
structure.calc = MatterSimCalculator(load_path=f"MatterSim-v1.0.0-{model}M.pth", device=device)

/usr/local/lib/python3.12/dist-packages/mattersim/forcefield/potential.py:896: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_path, map_location=

In [16]:
print(f"Energy (eV)                 = {structure.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {structure.get_potential_energy()/len(structure)}")
print(f"Forces of first atom (eV/A) = {structure.get_forces()[0]}")
print(f"Stress[0][0] (x-x) (eV/A^3) = {structure.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (x-x) (GPa)    = {structure.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -10.912513732910156
Energy per atom (eV/atom)   = -5.456256866455078
Forces of first atom (eV/A) = [-0.02334077  0.00218054  0.01819279]
Stress[0][0] (x-x) (eV/A^3) = 0.1328593064856631
Stress[0][0] (x-x) (GPa)    = 21.286407470703125


In [17]:
# initialize the relaxation object
relaxer = Relaxer(
    optimizer="BFGS", # the optimization method
    filter = "FrechetCellfilter",
    # filter="ExpCellFilter", # filter to apply to the cell
    constrain_symmetry=True, # whether to constrain the symmetry
)

relaxed_structure = relaxer.relax(structure, fmax=0.001, steps=500)

      Step     Time          Energy          fmax
BFGS:    0 06:47:00      -10.912512        3.663198
BFGS:    1 06:47:00      -11.506921        4.003321
BFGS:    2 06:47:00      -13.788439        3.171815
BFGS:    3 06:47:00      -14.625860        0.190155
BFGS:    4 06:47:00      -14.628172        0.196123
BFGS:    5 06:47:00      -14.184112        2.640507
BFGS:    6 06:47:00      -14.634958        0.221206
BFGS:    7 06:47:00      -14.641796        0.308700
BFGS:    8 06:47:00      -14.385571        2.258987
BFGS:    9 06:47:00      -14.655933        0.493703
BFGS:   10 06:47:00      -14.665016        0.618666
BFGS:   11 06:47:00      -14.667025        0.805115
BFGS:   12 06:47:00      -14.683677        0.695268
BFGS:   13 06:47:00      -14.692751        0.599783
BFGS:   14 06:47:00      -14.707533        0.337897
BFGS:   15 06:47:00      -14.714073        0.093146
BFGS:   16 06:47:00      -14.714493        0.040548
BFGS:   17 06:47:00      -14.714563        0.037339
BFGS:   18 06:

In [18]:
structure.get_positions()

array([[ 7.21726083e-03, -1.35003840e-03,  1.60664652e-03],
       [ 1.37044087e+00,  1.87825582e+00,  1.38471514e+00]])

In [19]:
print(f"Energy (eV)                 = {structure.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {structure.get_potential_energy()/len(structure)}")
print(f"Forces of first atom (eV/A) = {structure.get_forces()[0]}")
print(f"Stress[0][0] (x-x) (eV/A^3) = {structure.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (x-x) (GPa)    = {structure.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -14.962885856628418
Energy per atom (eV/atom)   = -7.481442928314209
Forces of first atom (eV/A) = [1.819586e-05 4.854519e-07 6.585188e-06]
Stress[0][0] (x-x) (eV/A^3) = 1.590344129916762e-05
Stress[0][0] (x-x) (GPa)    = 0.0025480121839791536


In [20]:
_ = visualise_structure(structure)

Structure positions: [[ 7.21726083e-03 -1.35003840e-03  1.60664652e-03]
 [ 1.37044087e+00  1.87825582e+00  1.38471514e+00]]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [21]:
structure.cell.cellpar()

array([ 2.74636027,  3.75921173,  2.74634662, 90.        , 89.99823972,
       90.        ])